# regrid the dataset to a healpix grid

Since the default geographic rectilinear grid has non-uniform grid sizes and distances and is thus not suitable for this application, we have to transform the dataset to one with equal distances between the cell centers. The Healpix cells satisfy these conditions, and can be used for spatial convolutions when reshaped to a 2D array according to the "nested" cell numbering.

As with any interpolation, the linear interpolation supported by the `healpy` library is split into the computation of weights and the application of those weights to the data.

In [ ]:
import xarray as xr
from pangeo_fish.healpy import create_grid, HealpyRegridder

parameters

In [ ]:
nside: int = 4096  # healpix resolution
rot = {"lat": 0, "lon": 30}


tag_name: str = "A19124"
working_path: str = "/home/datawork-taos-s/public/fish/"
# working_path: str | "/Users/todaka/python/git/pangeo-fish/data_local/fish-intel/"
ref_model_name: str = "copernicus"
cluster_size: int = 50
tag_base_path: str = "/home/datawork-taos-s/intranet/data/tag/Archival_Data/NetCDF/"
tag_db_path: str = "/home/datawork-taos-s/intranet/data/tag/Acoustic_Data/FishIntel_tagging_France.csv"
detections_path: str = "/home/datawork-taos-s/intranet/data/tag/Acoustic_Data/pollack_filtered_detections.csv"


In [ ]:

domainname=!domainname

if domainname == ["nisdatarmor"]:
    # Datarmor
#    tag_base_path = "/home/datawork-lops-iaocea/data/fish-intel/"
#    tag_base_path = "/home/datawork-taos-s/intranet/data/tag/"
    catalog = "/home/datawork-taos-s/intranet/kerchunk/ref-copernicus.yaml"
    cluster_name="datarmor"
else:
    # local PC
#    tag_base_path: str = "/Users/todaka/python/git/pangeo-fish/data_local/fish-intel/"
    catalog = "https://data-taos.ifremer.fr/kerchunk/ref-copernicus.yaml"
    cluster_name="local"

tag_url = tag_base_path +   tag_name + ".nc"

input_path = working_path + tag_name + "/" + ref_model_name + "/diff.zarr"
output_path = working_path + tag_name + "/" + ref_model_name + "/diff_"+ str(nside) +".zarr"


cluster

In [ ]:
import dask_hpcconfig
from distributed import Client

In [ ]:
if domainname == ["nisdatarmor"]:
    overrides = {}
    # overrides = { "cluster.cores": 28 , "cluster.processes": 6 }
    cluster = dask_hpcconfig.cluster("datarmor", **overrides)
    #    cluster = dask_hpcconfig.cluster("datarmor-local")
    cluster.scale(cluster_size)
else:
    cluster = dask_hpcconfig.cluster("local")

client = Client(cluster)
client

## read the data

In [ ]:
ds = xr.open_dataset(input_path, engine="zarr", chunks={})
ds

## define the target grid

In [ ]:
%%time
grid = create_grid(nside=nside, rot=rot)
grid

## compute the weights

In [ ]:
%%time
regridder = HealpyRegridder(ds, grid)
regridder

## apply the weights

In [ ]:
%%time
regridded = regridder.regrid_ds(ds)
regridded

## compute and save to disk

In [ ]:
regridded.to_zarr(output_path, mode="w", consolidated=True, compute=True)

## visualize the results

In [ ]:
regridded_ = xr.open_dataset(output_path, engine="zarr", chunks={})
regridded_

In [ ]:
regridded_["diff"].isel(time=0).plot(x="longitude", y="latitude")